In [1]:
import sys
sys.path.append("..")
import numpy as np
import keras.backend as K
import nea.asap_reader as dataset 
from nea.asap_reader import read_org_dataset
import nea.utils as U
import os
from time import time
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.visible_device_list = "0"
set_session(tf.Session(config=config))

Using TensorFlow backend.


In [2]:
train_path = '../../data/fold_0/train.tsv'
dev_path = '../../data/fold_0/dev.tsv'
test_path = '../../data/fold_0/test.tsv'
GLOVE_DIR = "../../../embeddings/glove/"
out_dir = 'output'
vocab_size = 4000
maxlen = 0
prompt_id = int(1)

In [3]:
from keras.preprocessing import sequence
# data_x is a list of lists
(train_x, train_y, train_pmt), (dev_x, dev_y, dev_pmt), (test_x, test_y, test_pmt), vocab, vocab_size, overal_maxlen, num_outputs = dataset.get_data(
(train_path, dev_path, test_path), prompt_id, vocab_size, maxlen, tokenize_text=True, to_lower=True, sort_by_len=False, vocab_path=None)

In [4]:
from keras.preprocessing import sequence
train_x = sequence.pad_sequences(train_x, maxlen=overal_maxlen)
dev_x = sequence.pad_sequences(dev_x, maxlen=overal_maxlen)
test_x = sequence.pad_sequences(test_x, maxlen=overal_maxlen)

In [5]:
train_y = np.array(train_y, dtype=K.floatx())
dev_y = np.array(dev_y, dtype=K.floatx())
test_y = np.array(test_y, dtype=K.floatx())
train_pmt = np.array(train_pmt, dtype='int32')
dev_pmt = np.array(dev_pmt, dtype='int32')
test_pmt = np.array(test_pmt, dtype='int32')
# We need the dev and test sets in the original scale for evaluation
dev_y_org = dev_y.astype(dataset.get_ref_dtype())
test_y_org = test_y.astype(dataset.get_ref_dtype())
# Convert scores to boundary of [0 1] for training and evaluation (loss calculation)
train_y = dataset.get_model_friendly_scores(train_y, train_pmt)
dev_y = dataset.get_model_friendly_scores(dev_y, dev_pmt)
test_y = dataset.get_model_friendly_scores(test_y, test_pmt)

In [6]:
from keras.models import Sequential, Model , Input
from keras.layers import Dense, Embedding, Activation , Flatten  , merge , BatchNormalization
from keras.layers.merge import Concatenate
from keras.layers import Dropout, GlobalAveragePooling1D ,GlobalMaxPooling1D , MaxPooling1D , AveragePooling1D , GlobalAveragePooling2D
from keras.layers import LSTM , Bidirectional , Convolution1D
from nea.optimizers import get_optimizer

In [54]:
loss = 'mean_squared_error'
metric = 'mean_absolute_error'
opt = get_optimizer('adam')

In [55]:
main_input = Input(shape=(overal_maxlen,), dtype='float64')
embedder = Embedding(len(vocab), 50, input_length = overal_maxlen  ) 
embed = embedder(main_input)
cnn1 = Convolution1D(filters=50 ,kernel_size=3)(embed)
cnn1 = MaxPooling1D(pool_size=3)(cnn1)
cnn1 = Dropout(0.2)(cnn1)

flat = Flatten()(cnn1)
#cnn1 = Convolution1D(filters=10 ,kernel_size=3)(embed)
#flat = Flatten()(cnn1)
main_output = Dense(1, activation='sigmoid')(flat)
model = Model(inputs = main_input, outputs = main_output)

In [56]:
loss = 'mean_squared_error'
metric = 'mean_absolute_error'
opt = get_optimizer('adam')

In [57]:
model.compile(loss=loss,
              optimizer=opt,
              metrics=[metric])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 939)               0         
_________________________________________________________________
embedding_11 (Embedding)     (None, 939, 50)           200000    
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 937, 50)           7550      
_________________________________________________________________
max_pooling1d_11 (MaxPooling (None, 312, 50)           0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 312, 50)           0         
_________________________________________________________________
flatten_11 (Flatten)         (None, 15600)             0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 15601     
Total para

In [58]:
from nea.evalute import Evaluator 
evl = Evaluator(dataset, prompt_id, out_dir, dev_x  ,test_x , dev_y, test_y, dev_y_org, test_y_org)
evl.evaluate(model, -1, print_info=True)

[Dev]   loss: 0.0447, metric: 0.1722, mean: 7.060 (8.508), stdev: 0.058 (1.539)
[Test]  loss: 0.0436, metric: 0.1752, mean: 7.061 (8.473), stdev: 0.054 (1.538)
[DEV]   QWK:  0.000, LWK: 0.000, PRS: -0.039, SPR: -0.008, Tau: 0.002 (Best @ -1: {{0.000}}, 0.000, -0.039, -0.008, 0.002)
[TEST]  QWK:  0.000, LWK: 0.000, PRS: 0.040, SPR: 0.030, Tau: 0.023 (Best @ -1: {{0.000}}, 0.000, 0.040, 0.030, 0.023)
--------------------------------------------------------------------------------------------------------------------------


In [59]:
total_train_time = 0
total_eval_time = 0
best , count = 0 , 0
for ii in range(100):
    # Training
    t0 = time()
    train_history = model.fit([train_x], train_y, batch_size=32, epochs=1, verbose=1)
    #train_history = model.fit([train_f], train_y, batch_size=32, epochs=1, verbose=1)
    tr_time = time() - t0
    total_train_time += tr_time
    # Evaluate
    t0 = time()
    evl.evaluate(model, ii)
    evl_time = time() - t0
    total_eval_time += evl_time
    train_loss = train_history.history['loss'][0]
    train_metric = train_history.history[metric][0]
    print('Epoch %d, train: %is, evaluation: %is' % (ii, tr_time, evl_time))
    print('[Train] loss: %.4f, metric: %.4f' % (train_loss, train_metric))
    evl.print_info()
    
evl.print_final_info()

Epoch 1/1
1070/1070 [==============================] - 1s 532us/step - loss: 0.0140 - mean_absolute_error: 0.0910
Epoch 0, train: 0s, evaluation: 0s
[Train] loss: 0.0140, metric: 0.0910
[Dev]   loss: 0.0081, metric: 0.0686, mean: 8.640 (8.508), stdev: 1.034 (1.539)
[Test]  loss: 0.0081, metric: 0.0696, mean: 8.614 (8.473), stdev: 1.029 (1.538)
[DEV]   QWK:  0.756, LWK: 0.546, PRS: 0.831, SPR: 0.826, Tau: 0.689 (Best @ 0: {{0.756}}, 0.546, 0.831, 0.826, 0.689)
[TEST]  QWK:  0.756, LWK: 0.563, PRS: 0.833, SPR: 0.813, Tau: 0.677 (Best @ 0: {{0.756}}, 0.563, 0.833, 0.813, 0.677)
--------------------------------------------------------------------------------------------------------------------------
Epoch 1/1
1070/1070 [==============================] - 0s 155us/step - loss: 0.0066 - mean_absolute_error: 0.0623
Epoch 1, train: 0s, evaluation: 0s
[Train] loss: 0.0066, metric: 0.0623
[Dev]   loss: 0.0067, metric: 0.0638, mean: 8.676 (8.508), stdev: 1.251 (1.539)
[Test]  loss: 0.0067, metric:

In [13]:
(0.827 + 0.794 + 0.824 + 0.836 + 0.834) /5 

0.8229999999999998

In [14]:
(0.674 + 0.65 + 0.645 + 0.647 + 0.685 ) / 5

0.6602

In [46]:
(0.65+ 0.663 + 0.660 + 0.704 + 0.668) / 5

0.6690000000000002

In [33]:
(0.787 + 0.795 + 0.767 +  0.773 + 0.762 )/5

0.7768

In [23]:
(0.8 + 0.821 + 0.798 + 0.767 + 0.819  )/5

0.8009999999999999

In [12]:
(0.78 + 0.796 + 0.796 + 0.789 + 0.785 )/5

0.7892

In [65]:
(0.748 + 0.756 + 0.788 + 0.789 +  0.771)/5

0.7704

In [147]:
(0.622 + 0.540 + 0.548 + 0.543 + 0.524)/5

0.5554

In [151]:
(0.8229 +0.6602 + 0.669+0.7768 + 0.8099+0.7892 + 0.7704 + 0.5554)/8

0.7317249999999998